In [ ]:
#%pip install google-generativeai

Defaulting to user installation because normal site-packages is not writeable
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/155.4 kB ? eta -:--:--
   ----------------------- ---------------- 92.2/155.4 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 155.4/155.4 kB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ------------- -------------------------- 0.4/1.3 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 13.8 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 14.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/173.7 kB ? eta -:--:--
   -------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.25.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.5 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
tf2onnx 1.16.1 requires protobuf~=3.20, but you have protobuf 5.29.5 which is incompatible.
tts 0.22.0 requires pandas<2.0,>=1.4, but you have pandas 2.1.3 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import google.generativeai as genai
import os
from dotenv import load_dotenv 
load_dotenv()

True

In [4]:
api_key = os.environ.get("gemini_api_key")

if not api_key:
    print("Error: gemini_api_key not found.")
    print("Please make sure you have a .env file with gemini_api_key=YOUR_KEY")
    exit()

In [5]:
import pandas as pd

df = pd.read_parquet('preprocessed_sample.parquet')

In [6]:
x = df[['mention_candidate_pairs', 'nil_examples', 'linked_examples']].head(1)

In [7]:
sample = pd.DataFrame(x['mention_candidate_pairs'].explode().apply(pd.Series)).head(5)

In [8]:
sample

,candidates,context,entity_type,mention,true_pageid,true_qid,true_title
0,[],EU rejects German call to boycott British lamb...,ORG,EU,NaN,NaN,None
0,[],EU rejects German call to boycott British lamb...,MISC,German,11867.0,183.0,Germany
0,[],EU rejects German call to boycott British lamb...,MISC,British,31717.0,145.0,United_Kingdom
0,[],EU rejects German call to boycott British lamb...,PER,Peter Blackburn,NaN,NaN,None
0,[],EU rejects German call to boycott British lamb...,LOC,BRUSSELS,3708.0,240.0,Brussels


In [9]:
prompt = f"Based on the Context {sample.iloc[0]['context']}, provide a brief, clarifying description for {sample.iloc[0]['mention']}. use English andprovide the answer directly without introductions"

In [10]:
try:
    genai.configure(api_key=api_key)
    
    model = genai.GenerativeModel('gemini-2.5-flash')

    for i in range(len(sample)):
        prompt = f"Based on the Context {sample.iloc[i]['context']}, provide a brief, clarifying description for {sample.iloc[i]['mention']}. use English and provide the answer directly without introductions (use 30 words reponse)"
        response = model.generate_content(prompt)

        print(f"{i+1}. Mention: {sample.iloc[i]['mention']}")
        print(response.text)
        print("\n")

except Exception as e:
    print(f"An error occurred: {e}")

1. Mention: EU
The EU, via its European Commission, acts as a supranational body that takes positions on trade and consumer advice impacting member states, here rejecting Germany's advice against British lamb.


2. Mention: German
Germany advised its consumers to boycott British lamb, citing scientific concerns. This call was specifically rejected by the EU Commission.


3. Mention: British
"British" refers to lamb originating from Britain (United Kingdom). The EU rejected Germany's call to boycott this lamb due to mad cow disease concerns, thus identifying its UK origin.


4. Mention: Peter Blackburn
Peter Blackburn was a reporter based in Brussels, covering the European Commission's disagreement with Germany's call for consumers to boycott British lamb in August 1996, amid concerns about mad cow disease.


5. Mention: BRUSSELS
BRUSSELS is the 1996 reporting location for Peter Blackburn. It serves as the administrative center for the European Commission, the body that rejected Germany

#### Testing with MedMentions Dataset

Now let's test the same clarification approach with biomedical entities from MedMentions.

In [11]:
# Load MedMentions data (from analysis - already flattened)
# Use a subset for testing
df_med = pd.read_json('data/processed/medmentions/train.jsonl', lines=True).head(100)

In [12]:
# Select interesting biomedical mentions (abbreviations, ambiguous terms)
med_sample = df_med[df_med['is_abbreviation'] == True].head(5)

if len(med_sample) == 0:
    # If no abbreviations in first 100, just take first 5
    med_sample = df_med.head(5)

med_sample[['mention', 'full_context', 'entity_type', 'is_abbreviation']]

,mention,full_context,entity_type,is_abbreviation
0,DCTN4,DCTN4 as a modifier of chronic Pseudomonas a...,"T116,T123",True
5,CF,DCTN4 as a modifier of chronic Pseudomonas aer...,T047,True
11,CPA,nfection in cystic fibrosis Pseudomonas aerugi...,T047,True
21,DCTN4,"unction, faster rate of lung decline, increase...","T116,T123",True
23,CF,increased rates of exacerbations and shorter ...,T047,True


In [15]:
# Generate clarifications for MedMentions entities

print("CLARIFY-AND-LINK: MedMentions Biomedical Entities")


try:
    for i, row in med_sample.iterrows():
        mention = row['mention']
        context = row['full_context']
        entity_type = row['entity_type']
        
        # Create prompt for biomedical entity clarification
        prompt = f"""Based on this biomedical context: "{context}"

Provide a brief, clarifying description for the medical term "{mention}".
Focus on what this term means in this specific medical context.
Use simple English and provide the answer directly without introductions (max 30 words)."""
        
        response = model.generate_content(prompt)
        
        print(f"\n Mention: '{mention}' (Type: {entity_type})")
        print(f" Clarification: {response.text}")
        print(f" Original context: ...{context[:100]}...")
        print("\n")

except Exception as e:
    print(f"An error occurred: {e}")

CLARIFY-AND-LINK: MedMentions Biomedical Entities

 Mention: 'DCTN4' (Type: T116,T123)
 Clarification: DCTN4 is a factor that influences (modifies) the severity or progression of chronic Pseudomonas aeruginosa infection in cystic fibrosis patients, impacting long-term lung disease.
 Original context: ... DCTN4  as a modifier of chronic Pseudomonas aeruginosa infection in cystic fibrosis Pseudomonas aer...



 Mention: 'DCTN4' (Type: T116,T123)
 Clarification: DCTN4 is a factor that influences (modifies) the severity or progression of chronic Pseudomonas aeruginosa infection in cystic fibrosis patients, impacting long-term lung disease.
 Original context: ... DCTN4  as a modifier of chronic Pseudomonas aeruginosa infection in cystic fibrosis Pseudomonas aer...



 Mention: 'CF' (Type: T047)
 Clarification: CF stands for Cystic Fibrosis, a genetic disease affecting patients whose lungs are highly susceptible to severe chronic Pseudomonas aeruginosa infections.
 Original context: ...DCTN4

### Comparison: AIDA vs MedMentions

**Key Differences:**

1. **Domain:**
   - AIDA: News articles (general entities like people, places, organizations)
   - MedMentions: Biomedical literature (diseases, procedures, chemicals, genes)

2. **Ambiguity Type:**
   - AIDA: Proper nouns with multiple meanings (Jordan = person, country, river)
   - MedMentions: Medical abbreviations and technical terms (MS = Multiple Sclerosis, Mass Spectrometry, etc.)

3. **Knowledge Base:**
   - AIDA: Wikipedia (general world knowledge)
   - MedMentions: UMLS (specialized medical ontology)

4. **Clarification Value:**
   - AIDA: Helps with world knowledge ambiguity
   - MedMentions: Provides expert medical knowledge clarification - **crucial for non-experts!**

**Why This Matters for Clarify-and-Link:**
- Shows the method works across **different domains**
- Demonstrates **generalization** beyond Wikipedia-based linking
- Proves value in **specialized/technical domains** where clarification is most needed